## ipcoal: simulation and analysis of genealogies and gene trees

The **ipcoal** Python package provides a convenient framework for simulating and analyzing genealogies and inferred gene trees under complex demographic scenarios. You can generate demographic models representing population histories, species trees, or networks, using a newick file that can be visualized in **toytree**. The model parameters will be parsed by **ipcoal** to define a simulation framework in **msprime** to generate a distribution of genealogies from which SNPs, loci, or chromosomes can be simulated. The simulated sequence data can be saved to disk in a variety of formats, or, gene tree analyses can be automated and parallelized to infer empirical gene trees on the simulated data. The resulting true genealogies, summary statistics, and inferred trees are returned by **ipcoal** is a Pandas DataFrame for further statistical analysis. 

### Required software
All required software can be installed with the following conda command. 

In [4]:
# conda install ipcoal -c eaton-lab conda-forge

In [5]:
import toytree
import ipcoal

### The main functions of *ipcoal*
You start by initializing a `Model` class object by providing a species tree/network and additional optional model parameters (e.g., Ne, migration, mutation rate, recombination rate). Then you can simulate either loci or SNPs on the genealogies produced under this model. **ipcoal** makes it easy to either write the sequence data to files under a variety of formats, or to perform phylogenetic inference on the sequence data directly. You can then compare true simulated genealogies to the inferred trees. Each of these functions is further demonstrated below.

In [3]:
# init a model Class object for simulations
tree = toytree.rtree.unittree(8, treeheight=1e6)
model = ipcoal.Model(tree=tree, Ne=1e6)

# simulate N unlinked SNPs (will run until N snps are produced)
model.sim_snps(100)

# simulate N loci of len L 
model.sim_loci(10, 300)

# view the genealogies and stats in a table
model.df

# save table to a CSV file
model.df.to_csv("./tree_table.csv")

# view sequence data as an array
model.seqs

# write loci as separate phylip files to a directory
model.write_loci_to_phylip(outdir="./tests")

# write concatenated loci or snps to a phylip file
model.write_concat_to_phylip(outdir="./tests", name="test.phy")

# infer a tree for every locus
model.infer_gene_trees(inference_method='raxml')

wrote 10 loci (8 x 300bp) to home/deren/Documents/physeqs/notebooks/tests/[...].phy
wrote concatenated loci (8 x 3000bp) to /home/deren/Documents/physeqs/notebooks/tests/test.phy


### Define a species/population tree
Node heights should be in units of generations. 

In [6]:
# generate a random 6-tip tree with root height of 1M generations
tree = toytree.rtree.coaltree(6, seed=222).mod.node_scale_root_height(1e6)

# draw tree showing idx labels
tree.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t30626ec1372d4e4ca03a06690e95860c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 87185.6719
support: 100
height: -0.0000 0 idx: 1
name: r1
dist: 87185.6719
support: 100
height: -0.0000 1 idx: 2
name: r2
dist: 171542.2928
support: 100
height: -0.0000 2 idx: 3
name: r3
dist: 171542.2928
support: 100
height: -0.0000 3 idx: 4
name: r4
dist: 389555.4181
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 389555.4181
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 504042.8998
support: 100
height: 87185.6719 6 idx: 7
name: 7
dist: 419686.2788
support: 100
height: 171542.2928 7 idx: 8
name: 8
dist: 408771.4283
support: 100
height: 591228.5717 8 idx: 9
name: 9
dist: 610444.5819
support: 100
height: 389555.4181 9 idx: 10
name: 10
dist: 195.7821
support: 100
height: 1000000.0000 10 0 500000 1000000

### Define an ipcoal simulation model. 
Here you can define the demographic model by setting a global Ne value (overrides Ne values stored to the tree), and setting the mutation and recombination rates. You can define a admixture scenarios using a simple syntax provided by a list of tuples. In each tuple you list the (source, dest, edge_prop, rate), where edge_prop is a float value of the proportion of the length of the shared edge between two taxa from recent to the past at which the migration pulse took place. In other words, if you set this to (7, 4, 0.5, 0.1) then 10% of the population of 7 will migrate into population 4 (backwards in time) at the midpoint of the shared edge between them. 

In [33]:
model = ipcoal.Model(
    tree,
    Ne=1e6,
    mut=1e-8,
    recomb=1e-9,
    seed=123,
    #admixture_edges=[(6, 4, 0.5, 0.1)],
)

In [35]:
model.sim_snps(1000)

In [36]:
mats = ipcoal.utils.get_snps_count_matrix(tree, model.seqs)

In [37]:
import toyplot
toyplot.matrix(mats[0], margin=0, width=300);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t1da27a3f368a46f587cee57221ae2eaf" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 300.0 300.0" width="300.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 85.000000 13.000000 6.000000 9.000000 6.000000 6.000000 0.000000 0.000000 16.000000 0.000000 6.000000 0.000000 5.000000 0.000000 0.000000 5.000000 1 15.000000 6.000000 0.000000 0.000000 5.000000 8.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 13.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 4.000000 0.000000 13.000000 0.000000 1.000000 0.000000 0.000000 0.000000 3 11.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 13.000000 4 7.000000 5.000000 0.000000 0.000000 4.000000 15.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 5 7.000000 5.000000 0.000000 1.000000 10.000000 77.000000 10.000000 11.000000 0.000000 13.000000 11.000000 0.000000 0.000000 10.000000 1.000000 5.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 12.000000 1.000000 0.000000 0.000000 0.000000 8.000000 1.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 1.000000 2.000000 0.000000 0.000000 3.000000 0.000000 0.000000 6.000000 0.000000 13.000000 8 14.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 14.000000 0.000000 0.000000 0.000000 1.000000 0.000000 9 0.000000 1.000000 0.000000 0.000000 0.000000 6.000000 2.000000 0.000000 0.000000 4.000000 9.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 8.000000 0.000000 11.000000 0.000000 0.000000 11.000000 18.000000 0.000000 12.000000 10.000000 78.000000 10.000000 0.000000 1.000000 10.000000 14.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9.000000 3.000000 0.000000 0.000000 5.000000 11.000000 12 7.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 6.000000 13 0.000000 0.000000 0.000000 0.000000 0.000000 9.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 10.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 1.000000 11.000000 2.000000 0.000000 0.000000 7.000000 16.000000 15 7.000000 0.000000 0.000000 15.000000 0.000000 8.000000 0.000000 10.000000 0.000000 0.000000 4.000000 8.000000 6.000000 12.000000 13.000000 67.000000

In [50]:
model.seqs.shape

(6, 100)

In [59]:
def abba_baba(tree, seqs):
    """
    Calculate ABBA/BABA statistic (D) as (ABBA - BABA) / (ABBA + BABA)
    """
    # get tip order of tree
    tips = [i for i in tree.get_tip_labels()]
    print(tips)
    
    # get counts matrix
    counts = ipcoal.utils.get_snps_count_matrix(tree, seqs)

    # store vals
    abbas = np.zeros(counts.shape[0], dtype=int)
    babas = np.zeros(counts.shape[0], dtype=int)
    dstats = np.zeros(counts.shape[0], dtype=float)
    p1 = np.zeros(counts.shape[0], dtype="U10")
    p2 = np.zeros(counts.shape[0], dtype="U10")
    p3 = np.zeros(counts.shape[0], dtype="U10")
    p4 = np.zeros(counts.shape[0], dtype="U10")

    # quartet iterator
    quarts = itertools.combinations(range(len(tips)), 4)

    # iterate over each (mat, quartet)
    idx = 0
    for count, qrt in zip(counts, quarts):

        # calculate
        abba = sum([count[i] for i in ABBA_IDX])
        baba = sum([count[i] for i in BABA_IDX])
        dstat = abs(abba - baba) / (abba + baba)

        # store stats
        abbas[idx] = abba
        babas[idx] = baba
        dstats[idx] = dstat

        # store taxa
        p1[idx] = tips[qrt[0]]
        p2[idx] = tips[qrt[1]]
        p3[idx] = tips[qrt[2]]
        p4[idx] = tips[qrt[3]]
        idx += 1

    # convert to dataframe   
    df = pd.DataFrame({
        "ABBA": np.array(abbas, dtype=int),
        "BABA": np.array(babas, dtype=int),
        "D": dstats,
        "p1": p1,
        "p2": p2,
        "p3": p3,
        "p4": p4,
        }, 
        columns=["ABBA", "BABA", "D", "p1", "p2", "p3", "p4"],
    )
    return df


In [11]:
model.seqs[[0, 1, 2, 3], :]

array([[1, 2, 1, ..., 2, 1, 2],
       [1, 2, 1, ..., 1, 0, 2],
       [1, 2, 2, ..., 1, 1, 2],
       [0, 2, 1, ..., 1, 1, 2]], dtype=uint8)

In [30]:
import itertools
import numpy as np
from ipcoal.jitted import count_matrix_int

# get matrix to fill
nquarts = sum(1 for i in itertools.combinations(range(tree.ntips), 4))
counts = np.zeros((nquarts, 16, 16), dtype=np.int64)

# iter
qiter = itertools.combinations(range(tree.ntips), 4)
quartidx = 0
for currquart in qiter:
    # cols indices match tip labels b/c we named tips node.idx
    quartsnps = model.seqs[currquart, :]
    
    # get counts
    counts[quartidx] = count_matrix_int(quartsnps.T)                    
    quartidx += 1

In [39]:
import toyplot
toyplot.matrix(counts[0], margi\n=0, width=350, height=350);

<svg class="toyplot-canvas-Canvas" height="350.0px" id="tcba6881c155c47bb9b63ff0dc395b7ab" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 350.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> DD 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 834.000000 103.000000 102.000000 102.000000 109.000000 78.000000 1.000000 1.000000 115.000000 0.000000 94.000000 0.000000 113.000000 1.000000 1.000000 82.000000 1 98.000000 35.000000 1.000000 1.000000 30.000000 87.000000 1.000000 1.000000 1.000000 0.000000 1.000000 0.000000 0.000000 3.000000 0.000000 0.000000 2 110.000000 1.000000 22.000000 0.000000 0.000000 0.000000 2.000000 0.000000 23.000000 0.000000 98.000000 0.000000 1.000000 0.000000 0.000000 0.000000 3 108.000000 1.000000 2.000000 17.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 28.000000 0.000000 0.000000 98.000000 4 102.000000 21.000000 0.000000 1.000000 18.000000 121.000000 2.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 1.000000 0.000000 2.000000 5 93.000000 95.000000 2.000000 1.000000 108.000000 797.000000 115.000000 104.000000 1.000000 109.000000 80.000000 1.000000 0.000000 116.000000 2.000000 64.000000 6 1.000000 1.000000 1.000000 0.000000 0.000000 119.000000 28.000000 1.000000 1.000000 27.000000 100.000000 2.000000 0.000000 0.000000 0.000000 0.000000 7 2.000000 0.000000 0.000000 0.000000 1.000000 116.000000 1.000000 26.000000 0.000000 0.000000 4.000000 0.000000 0.000000 32.000000 0.000000 99.000000 8 109.000000 1.000000 24.000000 2.000000 0.000000 1.000000 0.000000 0.000000 21.000000 0.000000 102.000000 0.000000 1.000000 0.000000 1.000000 2.000000 9 2.000000 1.000000 0.000000 0.000000 1.000000 104.000000 19.000000 0.000000 0.000000 32.000000 97.000000 0.000000 0.000000 0.000000 2.000000 1.000000 10 79.000000 0.000000 124.000000 0.000000 0.000000 98.000000 103.000000 0.000000 100.000000 89.000000 771.000000 121.000000 1.000000 2.000000 106.000000 84.000000 11 2.000000 0.000000 0.000000 1.000000 0.000000 3.000000 1.000000 0.000000 0.000000 1.000000 85.000000 37.000000 0.000000 0.000000 25.000000 117.000000 12 94.000000 1.000000 2.000000 24.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 1.000000 0.000000 19.000000 1.000000 2.000000 112.000000 13 3.000000 1.000000 0.000000 0.000000 0.000000 130.000000 2.000000 24.000000 0.000000 0.000000 1.000000 0.000000 0.000000 33.000000 0.000000 117.000000 14 2.000000 0.000000 0.000000 2.000000 0.000000 0.000000 1.000000 1.000000 3.000000 1.000000 106.000000 26.000000 2.000000 0.000000 36.000000 124.000000 15 78.000000 0.000000 0.000000 118.000000 1.000000 80.000000 1.000000 108.000000 1.000000 0.000000 77.000000 104.000000 93.000000 105.000000 123.000000 737.000000

### Simulate genealogies and sequences for N independent loci of length L
Because our simulation includes recombination each locus may represent multiple genealogical histories. You can see this in the dataframe below where loc 0 is represented by 5 genealogies. 

In [6]:
# run the simulation
model.sim_loci(nloci=10, nsites=500)

In [7]:
# view the genealogies and their summary stats
model.df.head(10)

,locus,start,end,nbps,nsnps,genealogy
0,0,0,104,104,7,"(r4:2.67611e+06,(r3:1.11..."
1,0,104,234,130,11,"((r0:2.20817e+06,r4:2.20..."
2,0,234,257,23,3,"((r4:2.20817e+06,(r0:1.3..."
3,0,257,438,181,20,"((r3:2.20817e+06,(r0:1.8..."
4,0,438,500,62,6,"((r3:2.20817e+06,r0:2.20..."
5,1,0,146,146,13,"((r2:1.3103e+06,r3:1.310..."
6,1,146,163,17,3,"((r1:840627,(r4:802207,r..."
7,1,163,175,12,2,"((r2:1.3103e+06,r3:1.310..."
8,1,175,212,37,2,"(r0:2.79004e+06,((r2:1.3..."
9,1,212,242,30,3,"((r1:840627,(r4:802207,r..."


### Visualize genealogical variation using toytree
Here the genealogies are plotted with tips in the same order as in the species tree so that you can easily identify the discordance between genealogies and the species tree. 

In [8]:
# load a multitree object from first 5 genealogies
mtre = toytree.mtree(model.df.genealogy)

# draw trees from the first locus
#  with 'shared_axis' to show diff in heights
#  with 'fixed_order' to show diff in topology (relative to first tree)
mtre.draw_tree_grid(
    start=0, ncols=4, nrows=1,
    shared_axis=True,
    fixed_order=tree.get_tip_labels(),
    tree_style='c', 
    node_labels=False, 
    node_sizes=8,
    tip_labels=True,
);

# draw trees from the second locus
mtre.draw_tree_grid(
    start=6, ncols=4, nrows=1,
    shared_axis=True,
    fixed_order=tree.get_tip_labels(),
    tree_style='c', 
    node_labels=False, 
    node_sizes=8,
    tip_labels=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t0b19524cfd774e63a2e68dce92d0019d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 250.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r0 r1 r2 r3 r4 r5 r0 r1 r2 r3 r4 r5 r0 r1 r2 r3 r4 r5 0 1338058 2676116

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t954098f525564b22ba94987bb25b2737" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 250.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r0 r1 r2 r3 r4 r5 r0 r1 r2 r3 r4 r5 r0 r1 r2 r3 r4 r5 0 1623036 3246073

### Write the simulated sequence data to file

In [9]:
# view the sequence array for the first locus (showing first 20 bp)
model.seqs[0, :, :20]

array([[1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 1, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3]],
      dtype=uint8)

In [10]:
# write all loci as separate phylip files to a directory
model.write_loci_to_phylip()

wrote 10 loci (6 x 500bp) to home/deren/Documents/physeqs/notebooks/ipcoal-sims/[...].phy


In [11]:
# write all loci concatenated to a single sequence file
model.write_concat_to_phylip()

wrote concatenated loci (6 x 5000bp) to /home/deren/Documents/physeqs/notebooks/test.phy


### Simulate N unlinked SNPs

In some cases you may only be interested in sampling unlinked SNPs. This is easy to do in **ipcoal** using the `.sim_snps()` function. This has two modes, the default is to simulate genealogies and attempt to drop a mutation on each one given the mutation rate. It will continue to generate new genealogies until you get the requested number of SNPs (which could take forever in some instances, like mutation_rate=0). The other option is to turn on the `repeat_on_trees=True` flag, which will continue to try to simulate a SNP on each tree until it is successful before moving on to the next tree. This may be slightly faster but will likely introduce biases. Only use the latter mode out of curiosity. 

When simulating SNPs the dataframe in `.df` is not particularly interesting, since every genealogy corresponds to only 1 site and 1 SNP. But it is still of interest for testing methods that rely on SNP data as a summary of the genealogy. The sequence data in `.seqs` is now a 2-d array (ntaxa, nsnps) as opposed to 3-d (nloci, ntaxa, nsites) when simulating loci. The functions to write the data to files works the same as before. You can call `.write_seqs_to_phy()` to write. 

In [12]:
# simulate N unlinked SNPs
model.sim_snps(100)

In [13]:
# the genealogies for each SNP are stored in .df
model.df.head()

,locus,start,end,nbps,nsnps,genealogy
0,0,0,1,1,1,"(r3:3.96605e+06,(r2:1.64..."
1,1,0,1,1,1,"((r1:1.38507e+06,(r0:693..."
2,2,0,1,1,1,"((r2:1.12575e+06,r5:1.12..."
3,3,0,1,1,1,"((r0:2.09737e+06,r2:2.09..."
4,4,0,1,1,1,"((r5:1.10614e+06,(r0:174..."


In [14]:
# the snp array is stored in .seqs
model.seqs[:, :20]

array([[0, 1, 3, 2, 3, 1, 3, 1, 0, 2, 3, 0, 3, 2, 2, 1, 2, 3, 1, 2],
       [0, 1, 3, 1, 3, 2, 1, 1, 0, 2, 3, 0, 3, 2, 2, 1, 2, 3, 1, 2],
       [0, 1, 3, 2, 2, 1, 1, 1, 1, 2, 3, 0, 3, 2, 0, 3, 2, 3, 1, 2],
       [2, 1, 3, 1, 2, 1, 1, 1, 0, 3, 2, 0, 0, 2, 0, 3, 2, 3, 1, 2],
       [0, 0, 3, 1, 2, 2, 1, 1, 1, 2, 3, 0, 3, 1, 1, 1, 0, 3, 1, 3],
       [0, 0, 1, 1, 3, 2, 1, 2, 1, 2, 3, 2, 3, 1, 0, 1, 0, 0, 0, 2]],
      dtype=uint8)

In [15]:
# write the snps array as a phylip file
model.write_concat_to_phylip()

wrote concatenated loci (6 x 100bp) to /home/deren/Documents/physeqs/notebooks/test.phy


### Infer gene trees 
Writing the sequence data to disk is optional and actually not required for some types of analyses, since *ipcoal* has built-in inference tools for inferring gene trees from the sequence data while it is stored in memory. This can create a really simple and reproducible workflow based simply on the random seed used for your analysis without a need to upload your simulated files to DRYAD at the end of your project. 

When you call one of the *inference* methods it will fill a new column in your dataframe called **inferred_trees**. 

In [16]:
# simulate locus data
tree = toytree.rtree.unittree(8, treeheight=1e6)
model = ipcoal.Model(tree=tree, Ne=1e5)
model.sim_loci(10, 500)

In [17]:
model.infer_gene_trees(inference_method="raxml")

In [18]:
model.df

,locus,start,end,nbps,nsnps,genealogy,inferred_tree
0,0,0,168,168,10,"((r7:883806,(r4:758813,r...",((r5:0.00401358103372166...
1,0,168,216,48,6,"(r0:1.5277e+06,((r7:8838...",((r5:0.00401358103372166...
2,0,216,277,61,4,"(r0:1.5277e+06,((r4:8838...",((r5:0.00401358103372166...
3,0,277,308,31,1,"((r0:815595,(r5:580343,r...",((r5:0.00401358103372166...
4,0,308,467,159,13,"((r5:882470,(r0:815595,r...",((r5:0.00401358103372166...
5,0,467,500,33,2,"((r2:519745,r1:519745):7...",((r5:0.00401358103372166...
6,1,0,131,131,11,"((r6:1.21659e+06,(r7:616...",(((r6:0.0162177841210500...
7,1,131,500,369,33,"((r6:1.21659e+06,(r7:616...",(((r6:0.0162177841210500...
8,2,0,500,500,25,"((r7:819768,r6:819768):2...",(((r1:0.0000010000005000...
9,3,0,73,73,11,"((r2:567261,r1:567261):8...",((r6:0.01011614698215194...


In [19]:
# save the dataframe with the inferred trees 
model.df.to_csv("./tree_table.csv")

### Write data as a site count matrix (*sensu* SVDquartets)

In [20]:
# for idx, mat in enumerate(snps.reshape((5,16,16))):
#     toyplot.matrix(mat, label="Matrix " + str(idx), colorshow=True);